***

# Turning Machine Simulator

### For csc427: Theory of Automata and Complexity. 
### University of Miami, Spring 2020.
### Burton Rosenberg.
__*Created: 21 March 2020*__
<br>__*Last update: 21 March 2020*__

***


### Turming Machine Description Syntax

The TM is described by a multiline string, with the format:

- **Comments:**
 - If the first character of the line after whitespace is #, the entire line is a comment.
- **Stanzas:**
 - Stanzas begin with a tag-name in column 1, a colon, and an argument.
 - Stanzas continue with a non-empty line beginning with whitespace.
- **Start:** 
 - Begins with tag "start". 
 - Argument is the start state.
 - There must be no continuation lines. 
- **Accept:**
 - Begins with tag "accept".
 - Argument is an accept state.
 - Each contination line, if any, is an accept state.
- **Reject:**
 - Begins with tag "reject".
 - Argument is an reject state.
 - Each contination line, if any, is an reject state.
- **State:**
 - Begin with the tag "state"
 - Argument is the state name that applies for the stanza
 - Each continuation line is a transition
 - Syntax of a continuation line:
   - *read_symbol write_symbol action state*.
 - Action must be one of l, r, or n, in either upper or lower case
 - If action is upper case, after performing the transtion, print the tape.
 - Unwritten tape is blank.
 - Use _ for write or read symbol the blank.
 - The symbol ":" reserved for left end of tape. 
 - Option "endmarker" will insert a ":" to the left end of the tape.
 - Left on the left end of the tape leaves the head position unchanged.


In [60]:
import string
import sys
import os
import argparse
import re

#
# tm-sim.py
#
# author: bjr
# date: 21 mar 2020
# last update: 22 mar 2020
#
# copyright: Creative Commons. See http://www.cs.miami.edu/home/burt
#


#
# BETA VERSION .. RELOAD OFTEN 
#

class TuringMachine:

    def __init__(self,verbose="none",endmarker=False):
        self.start_state = ""
        self.accept_states = set()
        self.reject_states = set()
        self.transitions = {}
        self.current_state = ""
        self.step_counter = 0
        self.all_actions = ["r","l","n"]
        self.verbose_levels = {"none":0, "verbose":1, "debug":2}
        self.tape = [' ']
        self.position = 0
        self.verbose = self.verbose_levels[verbose]
        self.endmarker = endmarker

    def set_start_state(self,state):
        self.start_state = state

    def set_tape(self,tape_string):
        # change '_' to ' '
        self.tape = [' ' if symbol=='_' else symbol 
                         for symbol in tape_string]
        if self.endmarker:
            self.tape.insert(0,':')
        
    def set_verbose(self,verbose):
        self.verbose = 0
        if verbose in self.verbose_levels:
            self.verbose = self.verbose_levels[verbose]

    def set_endmarker(self,endmarker):
        self.endmarker = endmarker

    def add_accept_state(self,state):
        self.accept_states.add(state)

    def add_reject_state(self,state):
        self.reject_states.add(state)
    
    def get_current_state(self):
        return self.curent_state

    def add_transition(self,state_from,read_symbol,
                       write_symbol,action,state_to):
        """
        Returns None on success; else return an error string.
        """
        
        if self.verbose >= self.verbose_levels['debug']:
            print("adding transition:", 
                  state_from, read_symbol, write_symbol, action, state_to )

        if read_symbol =='_': 
            read_symbol = ' '
        if write_symbol =='_':
            write_symbol = ' '

        if action.lower() not in self.all_actions:
            # return something instead, nobody likes a chatty program
            return "WARNING: unrecognized action, skipping."
        x = (state_from, read_symbol)
        if x in self.transitions:
            return "WARNING: multiple outgoing states not allowed for DFA's, skipping."
        self.transitions[x] = (state_to,write_symbol,action)
        return None

    def restart(self,tape_string):
        self.current_state = self.start_state
        self.position = 0
        if len(tape_string)==0 :
            tape_string = ' '
        self.set_tape(tape_string)
        self.step_counter = 1

    def step_transition(self):
        """
        take one state transition, based on tape, states, and transitions.
        Returns None if ok; else returns unmatched transition
        """
        c_s = self.current_state
        x = (c_s,self.tape[self.position])
        if x in self.transitions:
            (new_state, symbol, action ) = self.transitions[x]
        else:
            if self.verbose>=self.verbose_levels['debug']:
                print('current state:', c_s, 'current symbol: |', 
                      self.tape[self.position],'| current position: ', self.position)
            return str(x)
        self.current_state = new_state
        self.tape[self.position] = symbol

        shout = False
        if action.lower() != action:
            shout = True
            action = action.lower()
        
        if action == 'l' and self.position>0:
            self.position -= 1
        if action == 'r':
            self.position += 1
            if self.position==len(self.tape):
                self.tape[self.position:] = ' '
        if action == 'n':
            pass
   
        if shout:
            self.print_tape()

        if self.verbose >= self.verbose_levels['debug']:
            print("\t", self.step_counter, "\t", new_state, symbol, action)
        self.step_counter += 1
        return None

    def compute_tm(self,tape_string,step_limit=0):
        self.restart(tape_string)
        step = 0
            
        stop_states = self.accept_states.union(self.reject_states)
        while self.current_state not in stop_states:
            res = self.step_transition()
            if res:
                return ("no transition",res)
            step += 1
            if step > step_limit:
                return ("step limit",step,''.join(self.tape))
            if self.verbose >= self.verbose_levels['debug']:
                print(step, self.current_state, self.position, self.tape )

        cause = "reject"
        if self.current_state in self.accept_states:
            cause = "accept"
        the_tape = ''.join(self.tape)
        return (cause,the_tape)

    def print_tape(self):
        t, p = self.tape, self.position
        s = ''.join(t[:p] + ['_'] + [t[p]] + ['_'] + t[p+1:])
        print("step:",self.step_counter, "state:", self.current_state,"\t",s)
    
    def print_tm(self):
        print("\nstart state:\n\t",self.start_state)
        print("accept states:\n\t",self.accept_states)
        print("reject states:\n\t",self.reject_states)
        print("transitions:")
        for t in self.transitions:
            print("\t",t,"->",self.transitions[t])
        # print("tape:\n\t",self.tape)
        
### end class TuringMachine


class MachineParser:

    @staticmethod
    def turing(tm_obj, fa_string):
        """
        Code to parse a Turing Machine description into the Turing Machine object.
        """
        
        fa_array = fa_string.splitlines()
        line_no = 0 
        current_state = ""
        in_state_read = False
        in_accept_read = False
        in_reject_read = False

        for line in fa_array:
            while True:

                # comment lines are fully ignored
                if re.search('^\s*#',line):
                    break

                if re.search('^\s+',line):

                    if in_state_read:
                        m = re.search('\s+(\w|:)\s+(\w|:)\s+(\w)\s+(\w+)',line)
                        if m:
                            res = tm_obj.add_transition(current_state,
                                    m.group(1),m.group(2),m.group(3),m.group(4))
                            if res: 
                                print(res)
                            break

                    if in_accept_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_accept_state(m.group(1))
                            break

                    if in_reject_read:
                        m = re.search('\s+(\w+)',line)
                        if m:
                            tm_obj.add_reject_state(m.group(1))
                            break

                in_state_read = False
                in_accept_read = False
                in_reject_read = False

                # blank lines do end multiline input
                if re.search('^\s*$',line):
                    break ;

                m = re.search('^start:\s*(\w+)',line)
                if m:
                    tm_obj.set_start_state(m.group(1))
                    break

                m = re.search('^accept:\s*(\w+)',line)
                if m:
                    tm_obj.add_accept_state(m.group(1))
                    in_accept_read = True
                    break

                m = re.search('^reject:\s*(\w+)',line)
                if m:
                    tm_obj.add_reject_state(m.group(1))
                    in_reject_read = True
                    break

                m = re.search('^state:\s*(\w+)',line)
                if m:
                    in_state_read = True
                    current_state = m.group(1)
                    break

                print(line_no,"warning: unparsable line, dropping: ", line)
                break

            line_no += 1
        return

### end class MachineParser


def create_and_test_turing_machine(tm_description, test_cases, 
                                   verbose='none', endmarker=False):
    tm = TuringMachine(verbose,endmarker)
    MachineParser.turing(tm,tm_description)
    
    print("\n\n*** THE TURING MACHINE ***")
    tm.print_tm()

    print("\n\n*** TEST RUNS ***\n\n")

    for s in test_cases:
        print("input:",s)
        # assume complexity is some quadratic
        print(tm.compute_tm(s,step_limit=10*(len(s)+5)**2))
    print("\n\n*** RUN COMPLETE ***\n\n")


### Examples

In [55]:
tm1 = """# A-star-B-star:
# a*^b^*

start: s
accept: a
reject: r

state: s
    _ _ n a # accept the empty string
    a a r a_seen
    b b r b_seen

# any number of a's and end or some b's
state: a_seen
    a a r a_seen
    _ _ n a
    b b r b_seen
 
# any number of b's until end
state: b_seen
    b b r b_seen
    _ _ n a
    a a n r
"""

test_cases = [
    "",
    "aa",
    "bb",
    "ab",
    "ba",
    "aaabb",
    "aabbb",
    "abab"
]

create_and_test_turing_machine(tm1,test_cases)





*** THE TURING MACHINE ***

start state:
	 s
accept states:
	 {'a'}
reject states:
	 {'r'}
transitions:
	 ('s', ' ') -> ('a', ' ', 'n')
	 ('s', 'a') -> ('a_seen', 'a', 'r')
	 ('s', 'b') -> ('b_seen', 'b', 'r')
	 ('a_seen', 'a') -> ('a_seen', 'a', 'r')
	 ('a_seen', ' ') -> ('a', ' ', 'n')
	 ('a_seen', 'b') -> ('b_seen', 'b', 'r')
	 ('b_seen', 'b') -> ('b_seen', 'b', 'r')
	 ('b_seen', ' ') -> ('a', ' ', 'n')
	 ('b_seen', 'a') -> ('r', 'a', 'n')


*** TEST RUNS ***


input: 
('accept', ' ')
input: aa
('accept', 'aa ')
input: bb
('accept', 'bb ')
input: ab
('accept', 'ab ')
input: ba
('reject', 'ba')
input: aaabb
('accept', 'aaabb ')
input: aabbb
('accept', 'aabbb ')
input: abab
('reject', 'abab')


*** RUN COMPLETE ***




In [59]:
tm2="""# A simple context free language
# a^ib^i

start: s
accept: a
reject: r

# will use endmarker

state: s
    : : r s_1
    
state: s_1
    _ _ n a # accept the empty string
    a a r a_seen
    b b r b_seen
    
# any number of a's and some b's
state: a_seen
    a a r a_seen
    _ _ n r
    b b r b_seen
 
# any number of b's until end
state: b_seen
    b b r b_seen
    _ _ n syntax_done
    a a n r

state: syntax_done
    _ _ n find_b

# we proceed with the knowledge that at least to form is correct.
# we just need to count equal a's and b's

state: find_b
    _ _ l find_b
    x x l find_b
    b x l find_a
    a a n r   # too many a's
    : : n a   # everything has been crossed off

state: find_a
    b b l find_a
    x x l find_a
    a x r goto_endoftape
    : : n r   # too few a's

state: goto_endoftape
    a a r goto_endoftape
    b b r goto_endoftape
    x x r goto_endoftape
    _ _ n find_b

"""

test_cases = [
    "ab",
    "aabb",
    "abb",
    "aab"
]

create_and_test_turing_machine(tm2,test_cases,endmarker=True,verbose="verbose")



*** THE TURING MACHINE ***

start state:
	 s
accept states:
	 {'a'}
reject states:
	 {'r'}
transitions:
	 ('s', ':') -> ('s_1', ':', 'r')
	 ('s_1', ' ') -> ('a', ' ', 'n')
	 ('s_1', 'a') -> ('a_seen', 'a', 'r')
	 ('s_1', 'b') -> ('b_seen', 'b', 'r')
	 ('a_seen', 'a') -> ('a_seen', 'a', 'r')
	 ('a_seen', ' ') -> ('r', ' ', 'n')
	 ('a_seen', 'b') -> ('b_seen', 'b', 'r')
	 ('b_seen', 'b') -> ('b_seen', 'b', 'r')
	 ('b_seen', ' ') -> ('syntax_done', ' ', 'n')
	 ('b_seen', 'a') -> ('r', 'a', 'n')
	 ('syntax_done', ' ') -> ('find_b', ' ', 'n')
	 ('find_b', ' ') -> ('find_b', ' ', 'l')
	 ('find_b', 'x') -> ('find_b', 'x', 'l')
	 ('find_b', 'b') -> ('find_a', 'x', 'l')
	 ('find_b', 'a') -> ('r', 'a', 'n')
	 ('find_b', ':') -> ('a', ':', 'n')
	 ('find_a', 'b') -> ('find_a', 'b', 'l')
	 ('find_a', 'x') -> ('find_a', 'x', 'l')
	 ('find_a', 'a') -> ('goto_endoftape', 'x', 'r')
	 ('find_a', ':') -> ('r', ':', 'n')
	 ('goto_endoftape', 'a') -> ('goto_endoftape', 'a', 'r')
	 ('goto_endoftape', 'b')

### Exercise

Now recognize the non-CFL a^i b^i c^i, giving evidence that a Turing Machine encompasses and surpases both Regular Expressions and Context Free Languages.

